In [51]:
import pandas as pd
import requests
import json
from pathlib import Path

In [54]:
# Load existing data
old_eip_projects = pd.read_excel("../../../data/raw/2023.05.24 OGW database.xlsx", sheet_name = "Project")
transformed_data = pd.read_parquet('../../../data/output/data_warehouse/eip_facilities.parquet')

In [55]:
transformed_data

,facility_id,name,raw_created_on,raw_modified_on,raw_is_ccs,ccs_id,raw_company_id,raw_project_id,raw_state,facility_alias,...,unknown_id,is_ccs,state_id_fips,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county,longitude,latitude,date_modified
0,739,3 Bear Libby Gas Plant,2021-05-20T19:13:45.411472,2023-02-06T20:40:53.636126,True,5100.0,2447,2723,NM,<NA>,...,10000.0,True,35,35025,Lea,county,Lea,-103.525728,32.542358,2023-02-06 20:40:53.636
1,740,3 Brothers Compressor Station,2021-05-20T19:13:45.411472,2023-02-06T20:41:43.779072,<NA>,<NA>,2592,2724,PA,<NA>,...,10221.0,<NA>,42,42125,Washington,county,Washington,-80.380335,40.331198,2023-02-06 20:41:43.779
2,741,50 Buttes Gas Plant,2021-05-20T19:13:45.411472,2023-02-06T20:41:54.913539,<NA>,<NA>,2697,2725,WY,<NA>,...,10358.0,<NA>,56,56005,Campbell,county,Campbell,-105.779270,43.851240,2023-02-06 20:41:54.913
3,742,Agrium US Borger Nitrogen Operations,2021-05-20T19:13:45.411472,2023-02-07T02:43:55.022835,<NA>,<NA>,2624,2726,TX,<NA>,...,10266.0,<NA>,48,48113,Hutchins,city,Dallas County,-101.422777,35.641666,2023-02-07 02:43:55.022
4,743,Albany Compressor Station,2021-05-20T19:13:45.411472,2023-02-06T20:42:16.411585,<NA>,<NA>,2659,2729,GA,Albany Compressor Station-Southeast Market Pip...,...,10311.0,<NA>,13,13095,Dougherty,county,Dougherty,-84.250549,31.541712,2023-02-06 20:42:16.411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,6394,Lone Star CCS Hub,2023-05-22T14:36:02.875077,2023-05-23T16:02:19.584259,True,5036.0,"6395, 5362",6393,TX,<NA>,...,<NA>,True,48,<NA>,TX,,<NA>,NaN,NaN,2023-05-23 16:02:19.584
762,6396,Magnolia Sequestration Hub,2023-05-22T15:22:56.065726,2023-05-22T15:41:01.064250,True,<NA>,"6397, 2626",6398,LA,<NA>,...,<NA>,True,22,22003,Allen,county,Allen,NaN,NaN,2023-05-22 15:41:01.064
763,6400,Longleaf CCS Hub,2023-05-22T16:06:10.257152,2023-05-23T16:05:58.503996,True,<NA>,6401,6399,AL,<NA>,...,<NA>,True,01,01097,Mobile,county,Mobile,-88.014920,30.999682,2023-05-23 16:05:58.503
764,6402,SkyNRG Washington Plant,2023-05-22T17:24:01.492154,2023-05-22T17:47:34.647224,<NA>,<NA>,6403,6404,WA,<NA>,...,<NA>,<NA>,53,<NA>,WA,,<NA>,NaN,NaN,2023-05-22 17:47:34.647


## Get new data from API

The data is served through a xata API underlying the website. We query this directly as it provides access to the normalized data, and the new Excel file doesn't contain all the information we need.

In [4]:
eip_data_dictionary = {
    'facilities': '01-00_FACILITIES',
    'projects': '01-03_PROJECTS',
    'facility_project_assn': '01-03-0_link_Projects_Facilities',
    'air_construction_permits': '03-00_AIR_CONSTRUCTION',
    'air_construction_project_assn': '03-00-0_link_AirConstruction_Projects'
}

In [ ]:
# Set path for downloading new data for check
data_dir = Path().resolve()

PosixPath('/home/thinky/Desktop/Catalyst/PUDL/deployment-gap-model/notebooks/data_updates/eip_infrastructure')

In [6]:
def get_fossil_infrastructure_data(entity: str):
    dfs = []
    more = True
    body = {"page":{"size":999}}
    
    entity_endpoint = eip_data_dictionary[entity]
    while more is True: # While there are more pages to download
        response = requests.post(f"https://oilandgaswatch.org/xata-api/{entity_endpoint}/query", json = body)
        response_json = response.json()
        breakpoint()
        df = pd.DataFrame(response_json['records'])
        dfs.append(df)
        # Check for more records
        more = response_json['meta']['page']['more']
        body['page']['after'] = response_json['meta']['page']['cursor']

    df_combined = pd.concat(dfs).reset_index(drop = True)
    df_combined.to_csv(f"eip_{entity}_{pd.to_datetime('today').date()}.csv", index = False)

In [7]:
for key in eip_data_dictionary:
    get_fossil_infrastructure_data(entity = key)

Process the new data.

In [91]:
from dbcp.extract.eip_infrastructure import extract

raw_dfs = extract('/app/data/raw/eip_infrastructure')

In [92]:
facilities = raw_dfs['eip_facilities']

In [122]:
from dbcp.transform.eip_infrastructure import _format_column_names, _split_xata_column, _fix_erroneous_array_items


facilities = _split_xata_column(facilities)
facilities.columns = _format_column_names(facilities)

In [123]:
rename_dict = {  # add 'raw_' prefix to columns that need transformation
        "id": "facility_id",
        "updated_at": "raw_modified_on",
        "created_at": "raw_created_on",
        # "company_id": "raw_company_id",
        # "project_id": "raw_project_id",
        "state": "raw_state",
        "state_facility_id_numbers": "raw_state_facility_id_numbers",
        "primary_naics_code": "raw_primary_naics_code",
        "primary_sic_code": "raw_primary_sic_code",
        "street_address": "raw_street_address",
        "city": "raw_city",
        "countyor_parish": "raw_county_or_parish",
        "zip_code": "raw_zip_code",
        "countyor_parish": "raw_county_or_parish",
        "estimated_populationwithin3miles": "raw_estimated_population_within_3_miles",
        "percent_peopleof_colorwithin3miles": "raw_percent_people_of_color_within_3_miles",
        "percent_peopleof_color_percentile": "raw_percentile_people_of_color_within_3_miles",
        "percent_lowincomewithin3miles": "raw_percent_low_income_within_3_miles",
        "percent_lowincome_percentile": "raw_percentile_low_income_within_3_miles",
        "percent_under5years_oldwithin3miles": "raw_percent_under_5_years_old_within_3_miles",
        "percentunder5years_old_percentile": "raw_percentile_under_5_years_old_within_3_miles",
        "percent_peopleover64years_oldwithin3miles": "raw_percent_people_over_64_years_old_within_3_miles",
        "percentover64years_old_percentile": "raw_percentile_people_over_64_years_old_within_3_miles",
        "air_toxics_cancer_risk_nata_cancer_risk": "raw_air_toxics_cancer_risk_nata_cancer_risk",
        "air_toxics_cancer_risk_percentile": "raw_air_toxics_cancer_risk_percentile",
        # New columns
        "cancer_prevalence": "raw_cancer_prevalence",
        "cancer_prevalence_percentile": "raw_cancer_prevalence_percentile",
        "county_fips_code_text": "raw_county_fips_code",
        "location": "raw_location",
        "facility_footprint": "raw_facility_footprint",
        "epafrsid1": "raw_epa_frs_id_1",
        "epafrsid2": "raw_epa_frs_id_2",
        "epafrsid3": "raw_epa_frs_id_3",
        "id_qaqc": "unknown_id",
        "state_facility_id_numbers": "state_facility_id",
        

        # FIND THESE!
        # "associated_facilities_id": "raw_associated_facilities_id",
        # "pipelines_id": "raw_pipelines_id",
        # "air_operating_id": "raw_air_operating_id",
        # "latest_updates": "raw_latest_updates",
        # "cwa-npdes_id": "raw_cwa_npdes_id",
        # "cwa_wetland_id": "raw_cwa_wetland_id",
        # "other_permits_id": "raw_other_permits_id",
        # "congressional_representatives": "raw_congressional_representatives",
        # "ccs/ccus": "raw_is_ccs",
        # "respiratory_hazard_index": "raw_respiratory_hazard_index",
        # "pm2.5_ug/m3": "raw_pm2_5_ug_per_m3",
        # "o3_ppb": "raw_o3_ppb",
        # "wastewater_discharge_indicator": "raw_wastewater_discharge_indicator",
        
    }
facilities.rename(columns = rename_dict, inplace=True)

In [152]:
fac = raw_dfs['eip_facilities'].copy()
fac = _split_xata_column(fac)
fac.columns = _format_column_names(fac)
# Drop facilities that aren't published by EIP
fac = fac.loc[fac['published'] == True] 
rename_dict = {  # add 'raw_' prefix to columns that need transformation
    "id": "facility_id",
    "updated_at": "raw_modified_on",
    "created_at": "raw_created_on",
    # "company_id": "raw_company_id",
    # "project_id": "raw_project_id",
    "countyor_parish": "raw_county_or_parish",
    "state": "raw_state",
    "state_facility_id_numbers": "raw_state_facility_id_numbers",
    "primary_naics_code": "raw_primary_naics_code",
    "primary_sic_code": "raw_primary_sic_code",
    "street_address": "raw_street_address",
    "city": "raw_city",
    "zip_code": "raw_zip_code",
    "countyor_parish": "raw_county_or_parish",
    "estimated_populationwithin3miles": "raw_estimated_population_within_3_miles",
    "percent_peopleof_colorwithin3miles": "raw_percent_people_of_color_within_3_miles",
    "percent_peopleof_color_percentile": "raw_percentile_people_of_color_within_3_miles",
    "percent_lowincomewithin3miles": "raw_percent_low_income_within_3_miles",
    "percent_lowincome_percentile": "raw_percentile_low_income_within_3_miles",
    "percentunder5years_oldwithin3miles": "raw_percent_under_5_years_old_within_3_miles",
    "percentunder5years_old_percentile": "raw_percentile_under_5_years_old_within_3_miles",
    "percent_peopleover64years_oldwithin3miles": "raw_percent_people_over_64_years_old_within_3_miles",
    "percentover64years_old_percentile": "raw_percentile_people_over_64_years_old_within_3_miles",
    "air_toxics_cancer_risk_nata_cancer_risk": "raw_air_toxics_cancer_risk_nata_cancer_risk",
    "air_toxics_cancer_risk_percentile": "raw_air_toxics_cancer_risk_percentile",
    "location": "raw_location",
    "facility_footprint": "raw_facility_footprint",
    "epafrsid1": "raw_epa_frs_id_1",
    "epafrsid2": "raw_epa_frs_id_2",
    "epafrsid3": "raw_epa_frs_id_3",
    "id_qaqc": "unknown_id",
    # New columns
    "cancer_prevalence": "raw_cancer_prevalence",
    "cancer_prevalence_percentile": "raw_cancer_prevalence_percentile",
    "county_fips_code_text": "raw_county_fips_code",
    "county_fips_code_text": "raw_county_fips_code",
    "location": "raw_location",
    "facility_footprint": "raw_facility_footprint",
    "epafrsid1": "raw_epa_frs_id_1",
    "epafrsid2": "raw_epa_frs_id_2",
    "epafrsid3": "raw_epa_frs_id_3",
    "id_qaqc": "unknown_id",
    "state_facility_id_numbers": "state_facility_id",

    # FIND THESE!
    # "associated_facilities_id": "raw_associated_facilities_id",
    # "pipelines_id": "raw_pipelines_id",
    # "air_operating_id": "raw_air_operating_id",
    # "latest_updates": "raw_latest_updates",
    # "cwa-npdes_id": "raw_cwa_npdes_id",
    # "cwa_wetland_id": "raw_cwa_wetland_id",
    # "other_permits_id": "raw_other_permits_id",
    # "congressional_representatives": "raw_congressional_representatives",
    # "ccs/ccus": "raw_is_ccs",
    # "respiratory_hazard_index": "raw_respiratory_hazard_index",
    # "pm2.5_ug/m3": "raw_pm2_5_ug_per_m3",
    # "o3_ppb": "raw_o3_ppb",
    # "wastewater_discharge_indicator": "raw_wastewater_discharge_indicator",

}
fac.rename(columns=rename_dict, inplace=True)
should_be_numeric = [
    #"facility_id",
    "raw_estimated_population_within_3_miles",
    "raw_percent_people_of_color_within_3_miles",
    "raw_percentile_people_of_color_within_3_miles",
    "raw_percent_low_income_within_3_miles",
    "raw_percentile_low_income_within_3_miles",
    "raw_percent_under_5_years_old_within_3_miles",
    "raw_percentile_under_5_years_old_within_3_miles",
    "raw_percent_people_over_64_years_old_within_3_miles",
    "raw_percentile_people_over_64_years_old_within_3_miles",
    "raw_air_toxics_cancer_risk_nata_cancer_risk",
    "raw_air_toxics_cancer_risk_percentile",
    "raw_cancer_prevalence",
    "raw_cancer_prevalence_percentile",
    # "raw_respiratory_hazard_index",
    # "raw_pm2_5_ug_per_m3",
    # "raw_o3_ppb",
    # "raw_wastewater_discharge_indicator",
]
# Check that all columns that should be numeric are
for col in should_be_numeric:
    if not pd.api.types.is_numeric_dtype(fac[col]):
        fac[col] = pd.to_numeric(col, errors="raise")

# fix states that are CSV duplicates like "LA, LA"
# TODO: There are 3 facilities with multiple states - pick first
fac["state"] = _fix_erroneous_array_items(fac["raw_state"])
fac["state"].replace(["TBD", "TDB", ""], pd.NA, inplace=True)

# fix counties with multiple values
# Simplify by only taking first county. Only 11 multivalued as of 7/18/2023
fac["county"] = _fix_erroneous_array_items(
    fac["raw_county_or_parish"], split_on=",| and | or ", regex=True
)
fac["county"] = fac["county"].astype("string")
fac["county"].replace(["TBD", "TDB", "TBD County", "TBD Parish", ""], pd.NA, inplace=True)
# Strip leading and trailing whitespace
fac["county"] = fac["county"].str.strip()

# fix county FIPS codes with multiple values
# Simplify by only taking first FIPS code.
# 26 values as of Jan 9 2025.
fac["county_fips_code"] = _fix_erroneous_array_items(fac["raw_county_fips_code"])

# Geocode, then compare to EIP-provided data
fac = add_county_fips_with_backup_geocoding(
    fac, state_col="state", locality_col="county"
)
assert len(fac.loc[fac.county_id_fips != fac.county_fips_code]) == 0

fac.drop(columns=["state", "county"], inplace=True)  # drop intermediates

assert fac["longitude"].max() < 0  # USA longitudes
assert fac["latitude"].min() > 0  # USA latitudes

fac["date_modified"] = pd.to_datetime(
        fac.loc[:, "raw_modified_on"], infer_datetime_format=True
    )

In [155]:
len(fac.loc[fac.county_id_fips != fac["raw_county_fips_code"],['county_id_fips', 'raw_county_fips_code']])

26

In [156]:
from dbcp.transform.eip_infrastructure import facilities_transform

facilities_transform(raw_dfs['eip_facilities'])

ValueError: Unable to parse string "rec_cn781u7mpsqufoolsqb0" at position 896

In [106]:
from dbcp.transform.eip_infrastructure import add_county_fips_with_backup_geocoding

fac = add_county_fips_with_backup_geocoding(
        facilities, state_col="state", locality_col="county"
    )

KeyError: "None of [Index(['state', 'county'], dtype='object')] are in the [columns]"

In [20]:

old_eip_air = pd.read_excel("../../../data/raw/2023.05.24 OGW database.xlsx", sheet_name = "Project")
new_eip_air = pd.read_excel("../../../data/raw/2024.11.28 OGW database.xlsx", sheet_name = "Projects")

In [21]:
old_eip_air.columns

Index(['id', 'name', 'created_on', 'modified_on', 'Facility (ID)', 'Facility',
       'CCS (ID)', 'CCS', 'Project Description', 'Classification',
       'Industry Sector', 'Project Type', 'Product Type',
       'Air Construction (ID)', 'Air Construction', 'Air Operating (ID)',
       'Air Operating', 'NGA (ID)', 'NGA', 'MARAD (ID)', 'MARAD',
       'Other Permits (ID)', 'Other Permits', 'Greenhouse Gases (CO2e)',
       'Particulate Matter (PM2.5)', 'Nitrogen Oxides (NOx)',
       'Volatile Organic Compounds (VOC)', 'Sulfur Dioxide (SO2)',
       'Carbon Monoxide (CO)', 'Hazardous Air Pollutants (HAPs)',
       'Total Wetlands Affected Temporarily (acres)',
       'Total Wetlands Affected Permanently (acres)',
       'Detailed Permitting History', 'Emission Accounting Notes',
       'Construction Status Last Updated', 'Operating Status',
       'Actual or Expected Completion Year', 'Project Cost (million $)',
       'Number of Jobs Promised', 'Target List'],
      dtype='object')

In [13]:
new_eip_air.columns

Index(['State', 'County or Parish', 'Company', 'Facility',
       'Facility Description', 'Project', 'Operating Status',
       'Status Last Checked', 'Project Description', 'Project Classification',
       'Industry Sector', 'Project Type', 'Product Type',
       'CO2e PTE (tons/year)', 'PM2.5 PTE (tons/year)', 'NOx PTE (tons/year)',
       'VOC PTE (tons/year)', 'SO2 PTE (tons/year)', 'CO PTE (tons/year)',
       'HAPs PTE (lbs/year)', 'Latitude', 'Longitude', 'EJScreen Report Link',
       'Estimated population - 3 miles', 'Percent people of color - 3 miles',
       'Percent low income - 3 miles', 'ECHO Link 1', 'ECHO Link 2',
       'Company OGW ID', 'Facility OGW ID', 'Project OGW ID',
       'Facility OGW URL'],
      dtype='object')

In [5]:
import pandas as pd
transformed_data = pd.read_parquet('../../../data/output/data_warehouse/eip_air_constr_permits.parquet')

In [6]:
transformed_data.columns

Index(['air_construction_id', 'name', 'raw_created_on', 'raw_modified_on',
       'raw_date_last_checked', 'raw_project_id', 'raw_permit_status',
       'description_or_purpose', 'raw_application_date',
       'raw_draft_permit_issuance_date', 'raw_last_day_to_comment',
       'raw_final_permit_issuance_date', 'raw_deadline_to_begin_construction',
       'detailed_permitting_history', 'document_url', 'date_modified',
       'permit_status'],
      dtype='object')

In [6]:
old_eip

,id,name,created_on,modified_on,CCS/CCUS,CCS (ID),CCS,Company (ID),Company,Project (ID),...,Percent People over 64 Years Old within 3 miles,Air Toxics Cancer Risk (NATA Cancer Risk),Respiratory Hazard Index,PM2.5 (ug/m3),O3 (ppb),Wastewater Discharge Indicator,Location,Facility Footprint,EPA FRS ID,Facility ID
0,739,3 Bear Libby Gas Plant,2021-05-20T19:13:45.411472,2023-02-06T20:40:53.636126,True,5100.0,3 Bear Libby Gas Plant - CCS Project[5100],2447,"3 Bear Delaware Operating – NM, LLC[2447]",2723,...,15,20,0.20,7.32,55.1,2.400000e-11,POINT(-103.525728 32.542358),"POLYGON((-103.528453 32.540066,-103.528423 32....","110070159076, 110070789268",10000.0
1,740,3 Brothers Compressor Station,2021-05-20T19:13:45.411472,2023-02-06T20:41:43.779072,NaN,NaN,NaN,2592,"MarkWest Liberty Midstream & Resources, LLC[2592]",2724,...,25,38,0.30,8.44,44.2,3.000000e-02,POINT(-80.380335 40.331198),POINT(-80.380335 40.331198),110055589616,10221.0
2,741,50 Buttes Gas Plant,2021-05-20T19:13:45.411472,2023-02-06T20:41:54.913539,NaN,NaN,NaN,2697,"Thunder Creek Gas Services, LLC[2697]",2725,...,7,20,0.20,3.93,47.8,1.100000e-03,POINT(-105.77927 43.85124),NaN,110056978169,10358.0
3,742,Agrium US Borger Nitrogen Operations,2021-05-20T19:13:45.411472,2023-02-07T02:43:55.022835,NaN,NaN,NaN,2624,Agrium US Inc[2624],2726,...,14,20,0.21,6.51,52.0,2.300000e-03,POINT(-101.422777 35.641666),POINT(-101.422777 35.641666),110056955576,10266.0
4,743,Albany Compressor Station,2021-05-20T19:13:45.411472,2023-02-06T20:42:16.411585,NaN,NaN,NaN,2659,"Sabal Trail Transmission, LLC[2659]",2729,...,14,40,0.54,8.32,34.8,8.200000e-08,POINT(-84.250549 31.541712),NaN,110070119840,10311.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,6394,Lone Star CCS Hub,2023-05-22T14:36:02.875077,2023-05-23T16:02:19.584259,True,5036.0,BP & Linde CCS[5036],"6395, 5362","BP plc[6395], Linde Inc[5362]",6393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
762,6396,Magnolia Sequestration Hub,2023-05-22T15:22:56.065726,2023-05-22T15:41:01.064250,True,NaN,NaN,"6397, 2626","Magnolia Sequestration Hub LLC[6397], Occident...",6398,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
763,6400,Longleaf CCS Hub,2023-05-22T16:06:10.257152,2023-05-23T16:05:58.503996,True,NaN,NaN,6401,Tenaska[6401],6399,...,22,30,0.50,8.87,36.4,6.000000e-03,POINT(-88.01492 30.999682),NaN,NaN,NaN
764,6402,SkyNRG Washington Plant,2023-05-22T17:24:01.492154,2023-05-22T17:47:34.647224,NaN,NaN,NaN,6403,SkyNRG[6403],6404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
new_eip

,State,Company,Pipeline,Pipeline Description,Operating Status,Status Last Checked,Project Type,Product Type,Miles,Additional Capacity,Product Units,Associated Facilities,Associated Pipelines,Company OGW ID,Pipeline OGW ID,Associated Facilities OGW ID,Associated Pipelines OGW ID,Pipeline OGW URL
0,LA,Driftwood Pipeline LLC,Driftwood LNG Pipeline,The proposed Driftwood Pipeline Project would ...,Proposed,2022-02-16T00:00:00Z,New Pipeline,Natural Gas,99.4,3954.0,MMscf/d,"Basile Compressor Station, Woodside Louisiana ...","Driftwood Line 200 and Line 300 Project, Permi...",2506,3209,"755, 845, 880, 953","4199, 3348",https://oilandgaswatch.org/pipeline/3209
1,"IL, IN","Natural Gas Pipeline Company of America, LLC",134th Street Lateral Project,The 134th Street Lateral Project included a 1....,Operating,2022-06-16T00:00:00Z,Lateral,Natural Gas,1.4,70.0,MMscf/d,NaN,NaN,2613,3257,NaN,NaN,https://oilandgaswatch.org/pipeline/3257
2,MA,"Tennessee Gas Pipeline Company, LLC",261 Upgrade Projects,The 261 Upgrade Project delivers 72.4 million ...,Operating,2022-04-08T00:00:00Z,Expansion,Natural Gas,2.1,72.4,MMscf/d,Compressor Station 261,NaN,2689,3258,814,NaN,https://oilandgaswatch.org/pipeline/3258
3,LA,"Columbia Gulf Transmission, LLC, Kinder Morgan...",Acadiana Pipeline Project,The Acadiana Project increased firm north-to-s...,Operating,2022-04-14T00:00:00Z,Expansion,Natural Gas,0.0,894.0,MMscf/d,"Sabine Pass LNG Terminal, Eunice Compressor St...","Louisiana XPress Project, Driftwood Line 200 a...","2487, 3385",3259,"1047, 861","3319, 4199",https://oilandgaswatch.org/pipeline/3259
4,PA,Adelphia Gateway LLC,Adelphia Gateway Project,The Adelphia Gateway project converted the rem...,Operating,2023-09-14T00:00:00Z,Conversion,Natural Gas,4.7,250.0,MMscf/d,"Marcus Hook Compressor Station, Quakertown Com...",NaN,3386,3260,"3815, 3833",NaN,https://oilandgaswatch.org/pipeline/3260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,LA,Shell Oil Product US,Rome Pipeline,The Rome Pipeline is a proposed offshore pipel...,Announced,2024-11-21T17:18:50.045Z,New Pipeline,Oil,100.0,NaN,NaN,NaN,NaN,6180,rec_csvmmujt683r8q9cc030,NaN,NaN,https://oilandgaswatch.org/pipeline/rec_csvmmu...
310,"AL, GA, MS",Southern Natural Gas Company LLC,South System Expansion 4 (SSE4) Project,The South System Expansion 4 Project (SSE4) wo...,Proposed,2024-11-22T22:14:12.938Z,Expansion,Natural Gas,279.0,1200.0,MMscf/d,NaN,NaN,3405,rec_ct0g4dcke70h9fspkeu0,NaN,NaN,https://oilandgaswatch.org/pipeline/rec_ct0g4d...
311,TX,"EOG Resources, Inc.",South Texas Dorado-Verde Pipeline,Construction of 41 miles of new pipeline acros...,Operating,2024-11-25T22:12:43.33Z,New Pipeline,Natural Gas,41.0,1000.0,MMscf/d,NaN,NaN,rec_ct31g27rkv8v7ahcg4l0,rec_ct2fcmr6fvstbp7bv8p0,NaN,NaN,https://oilandgaswatch.org/pipeline/rec_ct2fcm...
312,TX,NaN,Seahawk Pipeline - Thunderdome Laterals,Converting two existing natural gas pipeline l...,Under Construction,2024-11-26T19:36:23.279Z,Conversion,Crude Oil,NaN,486000.0,bpd,NaN,NaN,NaN,rec_ct31os60g3jbpui8294g,NaN,NaN,https://oilandgaswatch.org/pipeline/rec_ct31os...


In [ ]:
import pandas as pd

from dbcp.helpers import get_sql_engine

engine = get_sql_engine()

with engine.connect() as con:
    new_acp_projects = pd.read_sql_table("fossil_infrastructure_projects", con, schema="data_mart")

In [19]:
githash = "a0fc9e0"
old_acp_projects = pd.read_parquet(f"../outputs/{githash}/dev/private_data_warehouse/acp_projects.parquet")

In [24]:
def pct_change_mw_by_status(new, old):

    new_agg = new.groupby("status").capacity_mw.sum()
    old_agg = old.groupby("status").capacity_mw.sum()
    return (new_agg - old_agg) / old_agg * 100

In [25]:
pct_change_mw_by_status(new_acp_projects, old_acp_projects)

status
Advanced Development      -11.403581
Decommissioned              2.578608
Online                      5.934032
Online | Decommissioned     0.416892
Under Construction          5.628867
Name: capacity_mw, dtype: float64

## % capacity change by status and ISO

In [30]:
new_acp_projects.iso_region.value_counts(dropna=False)

None         3588
MISO         1297
PJM          1193
CAISO        1072
ISONE         931
ERCOT         489
NYISO         418
SPP           379
WECC-RMRG       5
Name: iso_region, dtype: int64

In [31]:
for iso_region in new_acp_projects.iso_region.unique():
    new_prj_iso = new_acp_projects[new_acp_projects.iso_region == iso_region]
    old_prj_iso = old_acp_projects[old_acp_projects.iso_region == iso_region]

    pct_change = pct_change_mw_by_status(new_prj_iso, old_prj_iso)

    print(iso_region)
    print(pct_change)
    print()

CAISO
status
Advanced Development      -24.509428
Decommissioned              0.000000
Online                      4.616339
Online | Decommissioned     0.000000
Under Construction        -21.645777
Name: capacity_mw, dtype: float64

None
Series([], Name: capacity_mw, dtype: float64)

PJM
status
Advanced Development      -25.538129
Decommissioned            -22.468304
Online                      4.177580
Online | Decommissioned     0.000000
Under Construction         -3.123384
Name: capacity_mw, dtype: float64

ISONE
status
Advanced Development    0.859603
Decommissioned          0.000000
Online                  3.317459
Under Construction     -6.784373
Name: capacity_mw, dtype: float64

NYISO
status
Advanced Development   -7.567308
Online                  3.476287
Under Construction     -5.726114
Name: capacity_mw, dtype: float64

MISO
status
Advanced Development      -43.586332
Decommissioned              0.000000
Online                      3.340689
Online | Decommissioned     2.5840